# 4.4 自定义层

## 4.4.1 不含模型参数的自定义层

CenteredLayer类通过继承Module类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了forward函数里。这个层里不含模型参数。

In [1]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self,**kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self,x):
        return x-x.mean()

In [4]:
layer=CenteredLayer()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))#tensor类型，不加入模型参数中

tensor([-2., -1.,  0.,  1.,  2.])

In [25]:
#构造更复杂的模型

net=nn.Sequential(
    nn.Linear(8,128),
    CenteredLayer()
    )
print([param for param in net.parameters()])#???怎么写来着
out_put=net(torch.rand(4,8))
out_put.mean().item()

[Parameter containing:
tensor([[ 0.0307, -0.1503,  0.3277,  ..., -0.2909, -0.2300, -0.1935],
        [ 0.1517,  0.3018,  0.0679,  ...,  0.3481,  0.1872,  0.2509],
        [-0.3091,  0.0945, -0.3354,  ..., -0.3193, -0.1683,  0.2361],
        ...,
        [ 0.1305, -0.3437, -0.0086,  ..., -0.3305,  0.3207, -0.1169],
        [-0.1255, -0.2709, -0.1265,  ..., -0.3325, -0.3135, -0.1736],
        [ 0.1063,  0.2315, -0.1870,  ..., -0.3370,  0.2348, -0.1007]],
       requires_grad=True), Parameter containing:
tensor([ 0.0548, -0.0453,  0.1646, -0.2344,  0.0354,  0.2034,  0.0871, -0.3532,
        -0.3444,  0.0898, -0.0880, -0.2052,  0.3440,  0.1054, -0.1577,  0.0082,
        -0.3462,  0.3360,  0.1472,  0.1294, -0.0905,  0.1094, -0.0787, -0.0315,
        -0.0246,  0.1721,  0.2046, -0.2551, -0.3355,  0.2311,  0.1463, -0.3121,
        -0.0961,  0.0032, -0.2699, -0.1518,  0.2890,  0.1929,  0.1326, -0.2268,
        -0.3456,  0.2611, -0.1218,  0.2979,  0.2785,  0.0021, -0.2022, -0.0355,
         0.15

6.05359673500061e-09

## 4.4.2 含模型参数的自定义层

Parameter类其实是Tensor的子类，如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，我们应该将参数定义成Parameter，除了像4.2.1节那样直接定义成Parameter类外，还可以使用ParameterList和ParameterDict分别定义参数的列表和字典。

ParameterList接收一个Parameter实例的列表作为输入然后得到一个参数列表，使用的时候可以用索引来访问某个参数，另外也可以使用append和extend在列表后面新增参数。

In [34]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense,self).__init__()
        self.params=nn.ParameterList([nn.Parameter(torch.rand(4,4))for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))#追加参数
    
    def forward(self,x):
        for i in range(len(self.params)):
            x=torch.mm(x,self.params[i])
        return x

net=MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


而ParameterDict接收一个Parameter实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。例如使用update()新增参数，使用keys()返回所有键值，使用items()返回所有键值对等等

In [29]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params=nn.ParameterDict({
                'linear1':nn.Parameter(torch.randn(4,4)),
                'linear2':nn.Parameter(torch.randn(4,1))
        })
        self.params.update({
            'linear3':nn.Parameter(torch.randn(4,2))
        })
    
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.params[choice])

net=MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [30]:
x=torch.ones(1,4)
print(net(x,'linear1'))
print(net(x,'linear2'))
print(net(x,'linear3'))

tensor([[ 1.8807,  1.8962, -1.1041,  1.5257]], grad_fn=<MmBackward>)
tensor([[0.1889]], grad_fn=<MmBackward>)
tensor([[-0.0158, -1.4635]], grad_fn=<MmBackward>)


In [36]:
#使用自定义层构造模型

net=nn.Sequential(
    MyDictDense(),
    MyListDense(),
)
print(net)
print(net(x))

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[11.3014]], grad_fn=<MmBackward>)
